In [24]:
import os
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
from climatools.lblnew import setup_overlap
import climatools.lblnew.pipeline as pipeline_lblnew

importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(setup_overlap)

<module 'climatools.lblnew.setup_overlap' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/setup_overlap.py'>

In [25]:
def param_lblnew():
    return {'atmpro': 'mls',
 'band': '9',
 'commitnumber': '8ba2806',
 'dv': 0.001,
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'nv': 1000,
 'tsfc': 294}


def param_cliradlw():
    return {'atmpro': 'mls',
            'band': [11],
            'commitnumber': 'commitnumber', #'d54f896',
            'molecule': {'h2o': 'atmpro', 'co2': 0.0004},
            'tsfc': 294}


def parampairs():
    ps = []
    ps.append((param_cliradlw(), param_lblnew()))
    return ps


def script():
    ps = parampairs()
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps,
                                                 setup=setup)
    
    for param, _ in ps:
        print(pipeline.nbviewer_url(param=param, setup=setup))

In [26]:
script()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': 'commitnumber',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}

Submitting analysis for cases
/chia_cluster/home/jackyu/climatools/climatools/cliradlw/results_cliradlw.ipynb
/chia_cluster/home/jackyu/radiation/clirad-lw/LW/examples/lblnew/h2o_mls_co2_0.0004/band_11/atmpro_mls/cliradlw_commitnumber
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': 'commitnumber',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}

Committing analysis to Git repository for cases
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': 'commitnumber',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}

http://nbviewer.jupyter.org/github/qap/offline_radiation_notebooks/blob/master/longwave/lblnew_20160916/clirad/h2o_mls_co2_0.0004/band_11/atmpro_mls/cliradlw_commitnumber/results_cliradlw.ipynb
